# 事前処理

In [ ]:
import os
import sqlite3
import pandas as pd

class DataPrep():
  def __init__(self):
    # Set Variables
    self.file_name = "horse_race.sqlite"
    # Load SQLite
    self.con = sqlite3.connect(self.file_name)
    self.con_m = sqlite3.connect(':memory:')
    self.cur = self.con.cursor()
    self.cur_m = self.con_m.cursor()
    self.con.backup(self.con_m)    

  def load_from_file(self, race_date):
    df = self._load_x(race_date)
    # 全出走履歴の賞金合計
    df1 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize(x[0], x[1], x[2]), axis=1, result_type="expand")
    df2 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize(x[0], x[1], x[2]), axis=1, result_type="expand")
    df3 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize(x[0], x[1], x[2]), axis=1, result_type="expand")
    return df3

  def load_from_file_with_index(self, race_date):
    df = self._load_x(race_date)
    self.cur.execute("CREATE INDEX IF NOT EXISTS idx on HISTORY (horse_id, race_date)")
    # 全出走履歴の賞金合計
    df1 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize(x[0], x[1], x[2]), axis=1, result_type="expand")
    df2 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize(x[0], x[1], x[2]), axis=1, result_type="expand")
    df3 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize(x[0], x[1], x[2]), axis=1, result_type="expand")
    return df3

  def load_from_memory(self, race_date):
    # Load DB file to Memory
    df = self._load_x(race_date)
    # 全出走履歴の賞金合計
    df1 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize_m(x[0], x[1], x[2]), axis=1, result_type="expand")
    df2 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize_m(x[0], x[1], x[2]), axis=1, result_type="expand")
    df3 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize_m(x[0], x[1], x[2]), axis=1, result_type="expand")
    return df

  def load_from_memory_with_index(self, race_date):
    # Load DB file to Memory
    df = self._load_x(race_date)
    self.cur_m.execute("CREATE INDEX IF NOT EXISTS idx on HISTORY (horse_id, race_date)")
    # 全出走履歴の賞金合計
    df1 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize_m(x[0], x[1], x[2]), axis=1, result_type="expand")
    df2 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize_m(x[0], x[1], x[2]), axis=1, result_type="expand")
    df3 = df[["horse_id", "race_date", "type"]].progress_apply(lambda x: self.sum_prize_m(x[0], x[1], x[2]), axis=1, result_type="expand")
    return df

  def _load_x(self, date):
    sql = f'''
    SELECT
      t1.entry_id as entry_id,
      t1.race_id as race_id,
      t1.horse_id as horse_id,
      t1.jockey_id as jockey_id,
      t1.age as age,
      t1.burden as burden,
      t1.weight as weight,
      t1.weight_diff as weight_diff,
      t2.type as type,
      t2.race_date as race_date,
      t2.max_prize as max_prize,
      t3.father_id as father_id,
      t3.mother_id as mother_id,
      t3.f_father_id as father_id,
      t3.f_mother_id as mother_id
    FROM
      ENTRY t1
      LEFT JOIN RACE t2 on (t1.race_id = t2.race_id)
      LEFT JOIN HORSE t3 on (t1.horse_id = t3.horse_id)
    WHERE
      t2.race_date like '{date}%'
    '''
    return pd.read_sql(sql, self.con, index_col="entry_id")
  
  def sum_prize(self, horse_id, race_date, type, limit=999):
    sql = f'''
    SELECT 
      SUM(t1.prize), 
      AVG(t1.prize), 
      SUM(CASE WHEN t1.type == '{type}' THEN t1.prize ELSE 0 END), 
      AVG(CASE WHEN t1.type == '{type}' THEN t1.prize ELSE 0 END), 
      COUNT(*) 
    FROM (
      SELECT 
        prize, 
        type 
      FROM 
        HISTORY t1 
      WHERE
        t1.horse_id == '{horse_id}' AND 
        t1.race_date < '{race_date}'
      ORDER BY 
        race_date desc 
      LIMIT 
        {limit}) t1
      '''
    query = self.cur.execute(sql).fetchone()
    return query[0], query[1], query[2], query[3], query[4]

  def sum_prize_m(self, horse_id, race_date, type, limit=999):
    sql = f'''
    SELECT 
      SUM(t1.prize), 
      AVG(t1.prize), 
      SUM(CASE WHEN t1.type == '{type}' THEN t1.prize ELSE 0 END), 
      AVG(CASE WHEN t1.type == '{type}' THEN t1.prize ELSE 0 END), 
      COUNT(*) 
    FROM (
      SELECT 
        prize, 
        type 
      FROM 
        HISTORY t1 
      WHERE
        t1.horse_id == '{horse_id}' AND 
        t1.race_date < '{race_date}'
      ORDER BY 
        race_date desc 
      LIMIT 
        {limit}) t1
      '''
    query = self.cur_m.execute(sql).fetchone()
    return query[0], query[1], query[2], query[3], query[4]

  def version(self):
    return self.cur.execute('select sqlite_version()').fetchone()[0]

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook
from tqdm.notebook import tqdm
tqdm.pandas()
prep = DataPrep()

# SQLite3 Benchmark
SQLiteへの検索クエリ実行で、DBファイル `sqlite3.connect(file_name)` と インメモリ`sqlite3.connect(:memory:)` およびインデックスの有無でどれだけ差が出るのか確認したベンチマークになります。98,433件のテーブルに対して、複数の項目を使った条件抽出を含む集計クエリを24,006回実行し、どれくらいの時間で完了したのかを確認しています。

In [ ]:
print(f'SQLite Verison: {prep.version()}')

3.22.0


## Query for DB file
`con = sqlite3.connect(file_name)` に対するクエリ実行結果

In [ ]:
df = prep.load_from_file("2022")

  0%|          | 0/24006 [00:00<?, ?it/s]

  0%|          | 0/24006 [00:00<?, ?it/s]

  0%|          | 0/24006 [00:00<?, ?it/s]

## Query for DB file with Index
`con = sqlite3.connect(file_name)` および `cur.execute("CREATE INDEX IF NOT EXISTS idx on table (col_a, col_b)")` に対するクエリ実行結果

In [ ]:
df = prep.load_from_file_with_index("2022")

  0%|          | 0/24006 [00:00<?, ?it/s]

  0%|          | 0/24006 [00:00<?, ?it/s]

  0%|          | 0/24006 [00:00<?, ?it/s]

## Query for In Memory DB
`con = sqlite3.connect(:memory:)` に対するクエリ実行結果

In [ ]:
df = prep.load_from_memory("2022")

  0%|          | 0/24006 [00:00<?, ?it/s]

  0%|          | 0/24006 [00:00<?, ?it/s]

  0%|          | 0/24006 [00:00<?, ?it/s]

## Query for In Memory DB with Index
`con = sqlite3.connect(:memory:)` および `cur.execute("CREATE INDEX IF NOT EXISTS idx on table (col_a, col_b)")` に対するクエリ実行結果

In [ ]:
df = prep.load_from_memory_with_index("2022")

  0%|          | 0/24006 [00:00<?, ?it/s]

  0%|          | 0/24006 [00:00<?, ?it/s]

  0%|          | 0/24006 [00:00<?, ?it/s]